Test Aruco tag(aruco.DICT_4X4_50)

In [ ]:
import cv2
import numpy as np
# import cv2.aruco as aruco
from cv2 import aruco

#Aruco detection
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(0)

while True:
   ret, frame = cap.read()
   gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

   aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
   arucoParameters = aruco.DetectorParameters_create()
   corners, ids, rejectedImgPoints = aruco.detectMarkers(
       gray, aruco_dict, parameters=arucoParameters)
   print(ids)
   if np.all(ids):
     image = aruco.drawDetectedMarkers(frame,corners,ids)         
     cv2.imshow('Display', image)
   else:
       display = frame
       cv2.imshow('Display', display)
      
   if cv2.waitKey(1) & 0xFF == ord('q'):
       break

cap.release()
cv2.destroyAllWindows()

: 

Take phota

In [1]:
import cv2
import numpy as np
# import cv2.aruco as aruco
import time
cap = cv2.VideoCapture(0)
last_recorded_time = time.time()
frame_count = 0

while True: 
 _, frame = cap.read()
 curr_time = time.time()
 cv2.imshow('Display', frame)
 if (curr_time - last_recorded_time >= 2.0):
   cv2.imwrite('frame_{}.png'.format(frame_count), frame)
   last_recorded_time = curr_time
   frame_count+=1
 if cv2.waitKey(1) & 0xFF == ord('q'):
   break

cap.release()
cv2.destroyAllWindows()


Calibrate

In [3]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# size of each square is 35 mm
square_size = 0.018 #in meters
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = (square_size * np.mgrid[0:9,0:6]).T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# images = glob.glob(r'/home/pi/project/calib_photos/*.png',)
images = glob.glob(r'F:\System Folders\Desktop\RP_reference\RP_HW\project221020\*.png',)


for fname in images:
   img = cv2.imread(fname)
   gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
   sample_im = gray
   #cv2.imshow(gray)
   # Find the chess board corners
   chessboard_flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
   flags = 0
   flags |= cv2.CALIB_CB_ADAPTIVE_THRESH
   flags |= cv2.CALIB_CB_NORMALIZE_IMAGE
   [ret, corners] = cv2.findChessboardCorners(gray, (9, 6), flags)

   # If found, add object points, image points (after refining them)
   if (ret == True):
       #print ('True')
       objpoints.append(objp)
       corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
       imgpoints.append(corners2) #these are the found corners

       # Draw and display the corners
       img = cv2.drawChessboardCorners(img, (9,6), corners2,ret)
       cv2.imshow('img',img)
       cv2.waitKey(100)

cv2.destroyAllWindows()
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


# Undistortion
# refine the camera matrix
ii = -1
for fname in images:
   #if (fname==images[0]):
   ii +=1
   img = cv2.imread(fname)
   h,  w = img.shape[:2]
   newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
   dst1 = cv2.undistort(img, newcameramtx, dist, None, None)
  
   # testing if the matrix is right
   # image_path1 = r'/home/pi/project/calib_photos/final/%i.png' % ii
   image_path1 = r'F:\System Folders\Desktop\RP_reference\RP_HW\project221020\calib_photos\%i.png' % ii
   cv2.imwrite(image_path1, dst1)

# path = r'/home/pi/project/picamera.yml'  #F:\System Folders\Desktop\RP_reference\RP_HW\project221020\*.png
path = r'F:\System Folders\Desktop\RP_reference\RP_HW\project221020\picamera.yml' 
cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
cv_file.write("new_matrix", newcameramtx)
cv_file.write("distortion_coef", dist)

cv_file.release()

# Re-projection Error
tot_error = 0
for i in range(len(objpoints)):
   imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
   error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
   tot_error += error

print ("total error: ", tot_error/len(objpoints))

total error:  0.03713254932182226


In [4]:
import socket
import cv2
import numpy as np
import datetime
from cv2 import aruco
import glob
import math
import faulthandler; faulthandler.enable()
import multiprocessing as mp
import time

#IP = '192.168.0.246' #Emmett’s Robot IP
IP = '192.168.137.120' #Mahshid's Robot IP
port = 5000
marker_dimension = 0.046  #4.6 centimeter before
worldx = 889 #508#-marker_dimension*1000 #millimeters
worldy = 508 #401#-marker_dimension*1000 #millimeters
# modify these based on your enviornment settings
# robot_ID = 7
# bottom_left = 31  #this is the origin - positivex: towards bottom right - positivey: towards top left
# bottom_right = 32
# top_left = 9
# top_right = 20
robot_ID = 5
bottom_left = 1  #this is the origin - positivex: towards bottom right - positivey: towards top left
bottom_right = 2
top_left = 3
top_right = 4


# def UDP(IP,port,message):
#    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) #IPv4 DNS server - UDP protocol
#    sock.sendto(bytes(message, "utf-8"), (IP,port)) #self, data, address

def getMarkerCenter(corners):
 px = (corners[0][0] + corners[1][0] + corners[2][0]+ corners[3][0]) * 0.25
 py = (corners[0][1] + corners[1][1] + corners[2][1]+ corners[3][1]) * 0.25
 return [px,py]

def getMarkerRotation(corners):
 unit_x_axis = [1.,0.]
 center = getMarkerCenter(corners)
 right_edge_midpoint = (corners[1]+corners[2])/2.
 unit_vec = (right_edge_midpoint-center)/np.linalg.norm(right_edge_midpoint-center)
 angle = np.arccos(np.dot(unit_x_axis,unit_vec))
 return angle

def inversePerspective(rvec, tvec):
   R, _ = cv2.Rodrigues(rvec)
   R = np.array(R).T #this was np.matrix but had error
   invTvec = np.dot(-R, np.array(tvec))
   invRvec, _ = cv2.Rodrigues(R)
   return invRvec, invTvec
def normalize(v):
   if np.linalg.norm(v) == 0 : return v
   return v / np.linalg.norm(v)

"""" the function gets the corners of an aruco marker in the camera space as the origin with its
X (Green) and Y (Red) axis. The origin is the bottom left corner, the coordinate of the point is calculated
in relation to this origin """
def findWorldCoordinate(originCorners, point):
   zero = np.array(originCorners[3]) #bottom left as the origin - check the data structure
   x = (np.array(originCorners[0]) - zero)  # bottom right - Green Axis -- throw out z
   y = (np.array(originCorners[1]) - zero)   # top left - Red Axis -- throw out z
   x = x[0][0:2]
   y = y[0][0:2]
   x = normalize(x)
   y = normalize(y)
   #print("x", x)
   vec = (point - zero)[0][0:2]
   #print("vec", vec)
   vecNormal = normalize(vec)
   cosX = np.dot(x,vecNormal)
   cosY = np.dot(y,vecNormal)
   xW = np.linalg.norm(vec) * cosX
   yW = np.linalg.norm(vec) * cosY
   return [xW, yW]
################################
try:
   # Getting the camera calibration information
   path = r'/home/pi/ECE-5725/camera_codes/picamera.yml'
   cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)
   new_matrix = cv_file.getNode("new_matrix").mat()
   dist_matrix = cv_file.getNode("distortion_coef").mat()
   mtx = cv_file.getNode("matrix").mat()
   cv_file.release()

   found_dict_pixel_space = {}
   found_dict_camera_space = {}
   found_dict_world_space = {}
   found_dict_homography_space = {}
   final_string = ""
   originRvec = np.array([0,0,1])
   markerRvec= np.array([0,0,0])

 # read aruco markers and create dictionary
   aruco_source_path = glob.glob(r'/home/pi/ECE-5725/camera_codes/aruco/*.jpg', )
   aruco_source = []
   for ar in aruco_source_path:
       im_src = cv2.imread(ar)
       aruco_source.append(im_src)
   aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
  
   cap = cv2.VideoCapture(0)
   #pool = mp.Pool(mp.cpu_count()) # init pool for multiprocessing
  
   while True:
       t0 = time.time()
       #ret, frame = cap.read() # read frame
       ret, image = cap.read() # read frame
       #cv2.imshow('frame', frame)
       t1 = time.time()-t0
       #----aruco detection-------------------
       gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # converts image color space from BRG to grayscale
       t2 = time.time()-t0
       data = aruco.detectMarkers(gray, aruco_dict) # detect aruco markers
      
       t3 = time.time()-t0
       corners = data[0] # corners of found aruco markers
       ids = data[1] # ids of found aruco markers
       originIDglobal = 0
       if np.all(ids): # if any markers were found
           #print("found one!")
           t4 = time.time()-t0
           result = aruco.estimatePoseSingleMarkers(corners, marker_dimension, new_matrix, dist_matrix) # estimate the pose of the markers
           rvecs = result[0] # rotation vectors of markers
           tvecs = result[1] # translation vector of markers
           imageCopy = image

           #setting bottom_left as the origin
           if bottom_left in ids:
               originID = np.where(ids == bottom_left)[0][0]
               originIDglobal = originID
           else:
               originID = originIDglobal
           originCorners = corners[originID] # corners of the tag set as the origin
           OriginCornersCamera = getCornerInCameraWorld(marker_dimension, rvecs[originID], tvecs[originID])[0] # origin tag corners in camera space
           originRvec = rvecs[originID] # rotation vec of origin tag
           originTvec = tvecs[originID] # translation vec of origin tag
          
           display = cv2.aruco.drawDetectedMarkers(imageCopy,corners,ids) # display is image copy with boxes drawn around the tags
           t5 = time.time()-t0
           for i in range(len(ids)): # for each tag found in image
               ID = ids[i]
               rvec = rvecs[i]
               tvec = tvecs[i]
               corners4 = corners[i]
              
               display = cv2.aruco.drawAxis(imageCopy,new_matrix,dist_matrix,rvec,tvec,0.03) # draw 3d axis, 3 centimeters
               found_dict_pixel_space[""+str(ids[i][0])] = corners4 # put the corners of this tag in the dictionary
          
           # Homography
           zero = found_dict_pixel_space[str(bottom_left)][0][3] #bottom left - 3
           x = found_dict_pixel_space[str(bottom_right)][0][2] #bottom right - 27
           y = found_dict_pixel_space[str(top_left)][0][0] #top left - 22
           xy = found_dict_pixel_space[str(top_right)][0][1] #top right - 24

           workspace_world_corners = np.array([[0.0, 0.0], [worldx, 0.0], [0.0, worldy], [worldx, worldy]], np.float32) # 4 corners in millimeters
           workspace_pixel_corners = np.array([zero,x,y,xy], np.float32)  # 4 corners in pixels

           # Homography Matrix
           h, status = cv2.findHomography(workspace_pixel_corners, workspace_world_corners) #perspective matrix
           t6=time.time()-t0
           im_out = cv2.warpPerspective(image, h, (worldx,worldy)) #showing that it works
          
           t7 = time.time()-t0
           for i in range(len(ids)):
               j = ids[i][0]
               corners_pix = found_dict_pixel_space[str(j)]#[0]
               corners_pix_transformed = cv2.perspectiveTransform(corners_pix,h)
               found_dict_homography_space[str(j)] = corners_pix_transformed
           print(found_dict_homography_space)
           robot = found_dict_homography_space[str(robot_ID)][0]
           print(getMarkerCenter(robot))
           cv2.imshow('Warped Source Image', im_out)
           t8=time.time()-t0
           print("t1: %8.4f   t2: %8.4f   t3: %8.4f   t4: %8.4f   t5: %8.4f   t6: %8.4f   t7: %8.4f   t8: %8.4f" %(t1,t2-t1,t3-t2,t4-t3,t5-t4,t6-t5,t7-t6,t8-t7))
      
       else:
           display = image
           cv2.imshow('Display', display)

       ###### sending the data to the robot ########################
       robot_center = getMarkerCenter(robot)
       robot_angle = getMarkerRotation(robot)
       #UDP(IP, port, str([robot_center[0], robot_center[1],robot_angle,worldx,worldy]))
    #    UDP(IP, port, str([robot_center[0], robot_center[1], robot[1][0], robot[1][1], robot[0][0], robot[0][1], worldx, worldy]))

       if cv2.waitKey(1) & 0xFF == ord('q'):
           break

except Exception as e:
 print(e)

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

OpenCV(4.6.0) D:\bld\libopencv_1661643075477\work\modules\calib3d\src\calibration.cpp:1072: error: (-215:Assertion failed) CV_IS_MAT(objectPoints) && CV_IS_MAT(imagePoints) && CV_IS_MAT(A) && CV_IS_MAT(rvec) && CV_IS_MAT(tvec) in function 'cvFindExtrinsicCameraParams2'

